https://tekore.readthedocs.io/en/stable/reference/client.html#playlist-api


* Spotify.playlist_change_details: Change a playlist's details.
* Spotify.playlists: Get a list of the playlists owned or followed by a user.
* Spotify.playlist_cover_image: Get cover image of a playlist.
* Spotify.playlist_cover_image_upload: Upload a custom playlist cover image.
* Spotify.playlist_items: Full details of items on a playlist.


In [ ]:
import tekore as tk
conf = tk.config_from_environment()
token = tk.prompt_for_user_token(*conf)
spotify = tk.Spotify(token)


In [ ]:
import asyncio
import tekore as tk

async def fetch_playlists(spotify: tk.Spotify, userID: str):
    playlists = []
    inital_response = await spotify.playlists(userID, limit = 1, offset = 0)
    total = inital_response.total 
    playlists.extend(inital_response.items)
    limit = 10
    numOfCalls = (total - 1)//limit + 1
    
    tasks = [spotify.playlists(user_id=userID, limit = limit, offset = 1 + limit * i) for i in range(numOfCalls)]
    responses = await asyncio.gather(*tasks)
    for response in responses:
        playlists.extend(response.items)

    return playlists



In [ ]:
async_spotify = tk.Spotify(token, asynchronous=True)
a = await fetch_playlists(async_spotify, 'gingerale3')


In [ ]:
for i in range(len(a)):
    print({a[i].name, a[i].description, a[i].images[0].url})